### Dataset

Use Movielens dataset 100k movie ratings from 943 users and 1682 movies

In [19]:
import numpy as np
import math
import pandas as pd
import time

In [20]:
# Reading users file:
user_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
# users = pd.read_csv('../data/ml-100k/u.user', sep='|', names=user_cols, encoding='latin-1')

In [21]:
# Reading ratings file:
rating_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
# ratings = pd.read_csv('../data/ml-100k/u.data', sep='\t', names=rating_cols, encoding='latin-1')

In [22]:
# Reading items file
item_cols = ['movie_id', 'movie title', 'release data', 'video release date', 'IMDB URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drame', 'Fantasy', 'Film-Noir', 'Horror',
         'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
# items = pd.read_csv('../data/ml-100k/u.item', sep='|', names=item_cols, encoding='latin-1')

In [23]:
# dem so user va so item
n_users = 943
n_items = 1682
print("So user la: ", (n_users))
print("So bo phim la: ", (n_items))

So user la:  943
So bo phim la:  1682


In [24]:
# Bộ dữ liệu đã chia Movielen 100k đã chia săn thành 5 bộ dữ liệu trai-test 

# u1.base    -- The data sets u1.base and u1.test through u5.base and u5.test
# u1.test       are 80%/20% splits of the u data into training and test data.
# u2.base       Each of u1, ..., u5 have disjoint test sets; this if for
# u2.test       5 fold cross validation (where you repeat your experiment
# u3.base       with each training and test set and average the results).
# u3.test       These data sets can be generated from u.data by mku.sh.
# u4.base
# u4.test
# u5.base
# u5.test

In [25]:
# train_u1 = pd.read_csv('/home/damvantai/Documents/data/MovieLens/ml-100k/u1.base', sep='\t', names=rating_cols, encoding='latin-1')
# train_u2 = pd.read_csv('/home/damvantai/Documents/data/MovieLens/ml-100k/u2.base', sep='\t', names=rating_cols, encoding='latin-1')
# train_u3 = pd.read_csv('/home/damvantai/Documents/data/MovieLens/ml-100k/u3.base', sep='\t', names=rating_cols, encoding='latin-1')
# train_u4 = pd.read_csv('/home/damvantai/Documents/data/MovieLens/ml-100k/u4.base', sep='\t', names=rating_cols, encoding='latin-1')
train_u5 = pd.read_csv('/home/damvantai/Documents/data/MovieLens/ml-100k/u5.base', sep='\t', names=rating_cols, encoding='latin-1')

In [26]:
# test_u1 = pd.read_csv('/home/damvantai/Documents/data/MovieLens/ml-100k/u1.test', sep='\t', names=rating_cols, encoding='latin-1')
# test_u2 = pd.read_csv('/home/damvantai/Documents/data/MovieLens/ml-100k/u2.test', sep='\t', names=rating_cols, encoding='latin-1')
# test_u3 = pd.read_csv('/home/damvantai/Documents/data/MovieLens/ml-100k/u3.test', sep='\t', names=rating_cols, encoding='latin-1')
# test_u4 = pd.read_csv('/home/damvantai/Documents/data/MovieLens/ml-100k/u4.test', sep='\t', names=rating_cols, encoding='latin-1')
test_u5 = pd.read_csv('/home/damvantai/Documents/data/MovieLens/ml-100k/u5.test', sep='\t', names=rating_cols, encoding='latin-1')

### create matrix user-item rating

In [27]:
# tạo ma trận user-item, một cho training và một cho testing cho bo du lieu 2
# train_matrix_u2 = np.zeros((n_users, n_items))
# for line in train_u2.itertuples():
#     # line[1]: user_id
#     # line[2]: movie_id
#     # line[3]: rating
#     train_matrix_u2[line[1]-1, line[2]-1] = line[3]
    
# test_matrix_u2 = np.zeros((n_users, n_items))
# for line in test_u2.itertuples():
#     test_matrix_u2[line[1]-1, line[2]-1] = line[3]

# # tạo ma trận user-item, một cho training và một cho testing cho bo du lieu 3
# train_matrix_u3 = np.zeros((n_users, n_items))
# for line in train_u3.itertuples():
#     # line[1]: user_id
#     # line[2]: movie_id
#     # line[3]: rating
#     train_matrix_u3[line[1]-1, line[2]-1] = line[3]
    
# test_matrix_u3 = np.zeros((n_users, n_items))
# for line in test_u3.itertuples():
#     test_matrix_u3[line[1]-1, line[2]-1] = line[3]

# # tạo ma trận user-item, một cho training và một cho testing cho bo du lieu 4
# train_matrix_u4 = np.zeros((n_users, n_items))
# for line in train_u4.itertuples():
#     # line[1]: user_id
#     # line[2]: movie_id
#     # line[3]: rating
#     train_matrix_u4[line[1]-1, line[2]-1] = line[3]
    
# test_matrix_u4 = np.zeros((n_users, n_items))
# for line in test_u4.itertuples():
#     test_matrix_u4[line[1]-1, line[2]-1] = line[3]

# # tạo ma trận user-item, một cho training và một cho testing cho bo du lieu 1
train_matrix_u5 = np.zeros((n_users, n_items))
for line in train_u5.itertuples():
    # line[1]: user_id
    # line[2]: movie_id
    # line[3]: rating
    train_matrix_u5[line[1]-1, line[2]-1] = line[3]
    
test_matrix_u5 = np.zeros((n_users, n_items))
for line in test_u5.itertuples():
    test_matrix_u5[line[1]-1, line[2]-1] = line[3]


### Prediction and evaluate

Build matrix 

In [28]:
# User function pairwise_distance calculate simarility item-item and user-user
from sklearn.metrics.pairwise import pairwise_distances

### Function predict to calculate matrix pred

In [29]:
def predict(ratings, category, metric):
    
    # user-user matrix similarity
    if category == 'user':
        
        # Calculate matrix rating mean for user mean-user for rating item
        mean_rating = np.zeros([943, 1])
        for i in range(ratings.shape[0]):
            sum = 0
            num_elements = 0
            for j in range(ratings.shape[1]):
                if ratings[i][j] != 0:
                    sum = sum + ratings[i][j]
                    num_elements = num_elements + 1
            mean_rating[i] = float(sum) / num_elements
        
        # Create matrix filled with mean rating from above
        train_matrix_ratings_filled = np.array(ratings, copy=True)
        for i in range(ratings.shape[0]):
            for j in range(ratings.shape[1]):
                if train_matrix_ratings_filled[i][j] == 0:
                    train_matrix_ratings_filled[i][j] = mean_rating[i]
                           
        # Create matrix rating difference for matrix train filled and mean rating
        rating_diff = train_matrix_ratings_filled - mean_rating
        
        # Metric similarity cosine or correlation
        if metric == 'cosine':
            similarity = 1 - pairwise_distances(rating_diff, metric='cosine')
        elif metric == 'correlation':
            similarity = 1 - pairwise_distances(rating_diff, metric='correlation')
        
        # Initialization matrix predict 
        pred = np.zeros([n_users, n_items])
        
        # Calculate even element of matrix predict 
        # if rate of element identify, element predict unchange
        # else select 100 user similarity with user present and rating_diff != 0
        for i in range(n_users):
            for j in range(n_items):
                if ratings[i][j] != 0:
                    pred[i][j] = ratings[i][j]
                else:
#                     index_array_similarity_best = similarity[i].argsort()[-100:-1]
                    index_array_similarity_best = [user for user in similarity[i].argsort()[-100:-1] if rating_diff[user][j] != 0]
                    if (np.sum(similarity[i][index_array_similarity_best]) == 0):
#                         index_item_similary_best = [user for user in similarity[0].argsort()[-1000:-1] if rating_diff[user][111] != 0]
#                         pred[i][j] = np.sum(train_matrix_u1_filled.T[j])/n_items
                        # tinh rating cua item theo rating trung binh cua cac nguoi su dungj trong bang train cho item
#                         index_user_rating_item = np.nonzero(ratings.[i])
#                         pred[i][j] = np.sum(ratings[i][index_user_rating_item]) / np.shape(index_user_rating_item)[1]
                        pred[i][j] = mean_rating[i]
                        continue
                    pred[i][j] = similarity[i][index_array_similarity_best].dot(train_matrix_ratings_filled.T[j][index_array_similarity_best]) / np.sum(similarity[i][index_array_similarity_best])
            
        return pred

    # item-item similarity matrix
    elif category == 'item':
        
        # Initialization matrix mean for item mean for user
        mean_rating = np.zeros([1682, 1])
        
        # Create matrix item mean rating
        # except case a item don't have rate of user, default mean_rating of item equal 3
        for i in range(ratings.T.shape[0]):
            sum = 0
            num_elements = 0
            for j in range(ratings.T.shape[1]):
                if ratings.T[i][j] != 0:
                    sum = sum + ratings.T[i][j]
                    num_elements = num_elements + 1
            try:
                mean_rating[i] = float(sum) / num_elements
            except ZeroDivisionError:
                mean_rating[i] = 3
                
        # Calculate train matrix rating filled 
        train_matrix_ratings_filled = np.array(ratings.T, copy=True)
        for i in range(ratings.T.shape[0]):
            for j in range(ratings.T.shape[1]):
                if train_matrix_ratings_filled[i][j] == 0:
                    train_matrix_ratings_filled[i][j] = mean_rating[i]

        # matran rating difference
        rating_diff = train_matrix_ratings_filled - mean_rating
        if metric == 'cosine':
            item_similarity = 1 - pairwise_distances(rating_diff, metric='cosine')
        elif metric == 'correlation':
            item_similarity = 1 - pairwise_distances(rating_diff, metric='correlation')
        #         rating_diff = np.zeros([943, 1682])

        # Initialization matrix pred for item-item
        pred = np.zeros([n_items, n_users])

        # tinh toan ma tran pred theo cong thuc 
        for i in range(n_items):
            for j in range(n_users):
                if ratings.T[i][j] != 0:
                    pred[i][j] = ratings.T[i][j]
                else:
                    index_array_similarity_best = [user for user in item_similarity[i].argsort()[-100:-1] if rating_diff[user][j] != 0]
                    if (np.sum(item_similarity[i][index_array_similarity_best]) == 0):
                        pred[i][j] = mean_rating[i]
                        continue
                    pred[i][j] = item_similarity[i][index_array_similarity_best].dot(train_matrix_ratings_filled.T[j][index_array_similarity_best]) / np.sum(item_similarity[i][index_array_similarity_best])
    return pred.T

### Create list matrix train and test

### Function calculator error RMSE and MAE for element of matrix test

In [30]:
from sklearn.metrics.pairwise import pairwise_distances

In [31]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(predict, actual):
    predict = predict[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(predict, actual))

In [32]:
from sklearn.metrics import mean_absolute_error
def mae(predict, actual):
    predict = predict[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return (mean_absolute_error(predict, actual))

### Test user similarity cosine

In [33]:
# Want show full matrix numpy in ipython notebook

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
# # InteractiveShell.ast_node_interactivity = "last_expr"
# np.set_printoptions(threshold=np.nan)

### Run all train and test 

In [34]:
# matrix prediction user-user with similarity cosine
user_prediction = np.zeros([943, 1682])

# matrix prediction item-item with similarity cosine
item_prediction = np.zeros([943, 1682])

# matrix prediction item-item with similarity correlation
user_prediction_correlation = np.zeros([943, 1682])

In [35]:
# for i, matrix in enumerate(train_list):
user_prediction = predict(train_matrix_u5, category='user', metric='cosine')
item_prediction = predict(train_matrix_u5, category='item', metric='cosine')

#     print(user_prediction)

user_prediction_correlation = predict(train_matrix_u5, category='user', metric='correlation')
#     item_prediction_correlation = predict(matrix, item_similarity_correlation, type='item')

MAE_item = mae(item_prediction, test_matrix_u5)
MAE_user = mae(user_prediction, test_matrix_u5)
RMSE_item = rmse(item_prediction, test_matrix_u5)
RMSE_user = rmse(user_prediction, test_matrix_u5)

#     RMAE_item_correlation[i] = rmse(item_prediction_correlation, test_list[i])
RMSE_user_correlation = rmse(user_prediction_correlation, test_matrix_u5)
print('{0:.5f} {1:.5f} {2:.5f} {3:.5f} {4:.5f}'.format(
    MAE_user, RMSE_user, MAE_item, RMSE_item, 
    RMSE_user_correlation))    

0.80900 1.02421 0.77737 1.00028 1.02421


In [36]:
print(RMSE_item)

1.0002830775412475


In [37]:
print(RMSE_user)

1.0242113814633078


### Collaborative based model